# All modules fast check 

In [1]:
import main_function as mf


## Set batch name and get data form files
1. 'old' last batch of cells
2. 'Dec' YiHui's new cells from Dec
3. 'gold-copper' Single junction cells with gold/copper electrode
4. 'Jan' New cells in Jan


In [2]:
#mf.batch_name = 'Old'
#mf.batch_name = 'Dec'
#mf.batch_name = 'gold-copper'
mf.batch_name = 'Jan'

data = mf.read_all_test()

## Draw overall PCE chart

In [3]:
mf.draw_cell_PCE_overview(data)

## Draw overall PCE ratio chart

In [4]:
mf.draw_cell_PCE_overview_ratio(data)

## Draw all cell performance change ratio

In [5]:
mf.draw_cell_performance(data)

,Hours,Voc_r,Jsc_r,FF_r,PCE_r,Voc_f,Jsc_f,FF_f,PCE_f
0,0.0,1.755891,17.155339,0.844362,25.434657,1.757308,17.151181,0.844731,25.460126


,Hours,Voc_r,Jsc_r,FF_r,PCE_r,Voc_f,Jsc_f,FF_f,PCE_f
0,0.0,1.736720,18.303654,0.865547,27.514300,1.730446,18.231614,0.846878,26.718010
1,24.0,0.803531,22.423044,0.718878,12.952454,0.833968,22.330491,0.704157,13.113452


,Hours,Voc_r,Jsc_r,FF_r,PCE_r,Voc_f,Jsc_f,FF_f,PCE_f
0,0.0,1.707539,16.863525,0.795684,22.911818,1.716856,16.980711,0.748063,21.808616
1,24.0,1.744954,16.182517,0.820425,23.166968,1.743000,16.125232,0.778046,21.867984
2,48.0,1.760494,17.114459,0.790512,23.818056,1.761178,17.011157,0.735970,22.049437


,Hours,Voc_r,Jsc_r,FF_r,PCE_r,Voc_f,Jsc_f,FF_f,PCE_f
0,0.0,1.753282,16.657488,0.851528,24.869113,1.763732,17.651579,0.700507,21.808643
1,24.0,1.790859,16.161779,0.857441,24.817316,1.790220,16.071716,0.825167,23.741646
2,48.0,1.801377,16.917667,0.857470,26.131475,1.793954,16.886185,0.813405,24.640509


,Hours,Voc_r,Jsc_r,FF_r,PCE_r,Voc_f,Jsc_f,FF_f,PCE_f
0,0.0,1.751604,21.889897,0.752820,28.864964,1.749043,21.911930,0.759663,29.114033
1,24.0,1.755622,19.877217,0.781366,27.267255,1.760908,20.053853,0.771118,27.230464
2,48.0,1.763720,20.920296,0.776594,28.654413,1.753030,20.761242,0.769480,28.005278
